# Register Model

## Notebook Overview

- Start Execution
- Install and Import Libraries
- Configure Settings
- Register and Log the Model to MLFlow

# Start Execution

In [1]:
import logging
import time

# Configure logger
logger: logging.Logger = logging.getLogger("register_model_logger")
logger.setLevel(logging.INFO)
logger.propagate = False  # Prevent duplicate logs from parent loggers

# Set formatter
formatter: logging.Formatter = logging.Formatter(
    fmt="%(asctime)s - %(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S"
)

# Configure and attach stream handler
stream_handler: logging.StreamHandler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)

In [2]:
start_time = time.time()  

logger.info("Notebook execution started.")

2025-08-13 19:42:40 - INFO - Notebook execution started.


# Install and Import Libraries

In [3]:
%%time

%pip install -r ../requirements.txt --quiet 

Note: you may need to restart the kernel to use updated packages.
CPU times: user 14.5 ms, sys: 153 μs, total: 14.7 ms
Wall time: 941 ms


In [4]:
import os
import sys

#os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
from src.utils import (
    load_configuration,
    load_secrets,
    load_secrets_to_env,
    configure_proxy,
    get_project_root,
    get_config_dir,
    get_output_dir,
    get_default_model_path,
    get_model_cache_dir,
    setup_dreambooth_model
)


# Add the core directory to the path to import utils
core_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
if core_path not in sys.path:
    sys.path.append(core_path)

from core.custom_metrics.image_metrics_scorers import entropy_scorer, complexity_scorer, set_custom_image_path
from core.deploy.deploy_image_generation import deploy_model
from core.local_inference.inference import StableDiffusionPipelineOutput, load_config, run_inference
from core.dreambooth_inference.inference_dreambooth import StableDiffusionPipelineOutput, load_config_dreambooth, run_inference_dreambooth


# === Third-Party Imports ===
from huggingface_hub import snapshot_download
import glob
import torch


# === Standard Library Imports ===
import logging
import warnings
from pathlib import Path
from typing import Dict, Any, Optional, Union, List, Tuple


2025-08-13 19:42:42.437319: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-13 19:42:42.458961: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755114162.469418    1276 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755114162.472378    1276 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755114162.483070    1276 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

# Configure Settings

In [5]:
# ------------------------ Suppress Verbose Logs ------------------------
warnings.filterwarnings("ignore")

In [6]:
# === Constants ===
# Model and experiment configuration
project_root = get_project_root()
CONFIG_PATH = "../configs/config.yaml"
SECRETS_PATH = "../configs/secrets.yaml"


In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


# Register and Log the Model to MLFlow 

Using MLflow, we will save and load the model in an integrated manner, enabling the traceability and reproducibility of experiments. MLflow will facilitate model versioning, monitoring, and deployment, ensuring a robust pipeline for your project.

In [8]:
%%time
deploy_model()

2025-08-13 19:42:49,330 — INFO — Using accelerate cfg: /home/jovyan/AI-Blueprints/generative-ai/image-generation-with-stablediffusion/configs/default_config_one-gpu.yaml
2025-08-13 19:42:49,516 — INFO — Starting model deployment...
2025-08-13 19:42:49,532 — INFO — Using local base model: /home/jovyan/AI-Blueprints/generative-ai/image-generation-with-stablediffusion/models/stable-diffusion-2-1
2025-08-13 19:42:49,533 — INFO — Using finetuned model: /home/jovyan/AI-Blueprints/generative-ai/image-generation-with-stablediffusion/output/dreambooth
2025-08-13 19:42:49,533 — INFO — Using base model: /home/jovyan/AI-Blueprints/generative-ai/image-generation-with-stablediffusion/models/stable-diffusion-2-1
2025-08-13 19:42:49,673 — INFO — 📦 Logging artifacts and model...
2025/08/13 19:42:49 INFO mlflow.models.signature: Inferring model signature from type hints
/opt/conda/lib/python3.12/site-packages/mlflow/pyfunc/__init__.py:3139: UserWarning: Failed to infer signature from type hint: Type hin

2025-08-13 19:47:43,720 — INFO — ✅ Model logged to MLflow at 'image_generation_model'
Registered model 'ImageGenerationService' already exists. Creating a new version of this model...
Created version '3' of model 'ImageGenerationService'.
2025-08-13 19:47:44,084 — INFO — 🏷️ Registered 'ImageGenerationService' (run 59fb08cc75ef44b69ac697dd1b0da567)
2025-08-13 19:47:44,084 — INFO — Model deployment completed successfully


CPU times: user 1.28 s, sys: 33.4 s, total: 34.7 s
Wall time: 4min 55s


In [9]:
end_time: float = time.time()
elapsed_time: float = end_time - start_time
elapsed_minutes: int = int(elapsed_time // 60)
elapsed_seconds: float = elapsed_time % 60

logger.info(f"⏱️ Total execution time: {elapsed_minutes}m {elapsed_seconds:.2f}s")
logger.info("✅ Notebook execution completed successfully.")

2025-08-13 19:47:44 - INFO - ⏱️ Total execution time: 5m 3.52s
2025-08-13 19:47:44 - INFO - ✅ Notebook execution completed successfully.


Built with ❤️ using [**HP AI Studio**](https://hp.com/ai-studio).